In [1]:
# Load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np
from sqlalchemy import create_engine
engine = create_engine("sqlite:///master_db.sqlite",echo=False)

In [2]:
exhist = pd.read_sql_query('select * from exhist',con=engine)
exhist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689982 entries, 0 to 689981
Data columns (total 13 columns):
extract_point            689982 non-null object
extraction_tons          689982 non-null int64
datestring               689982 non-null int64
total_extraction_tons    689982 non-null int64
reserves                 689982 non-null float64
region                   689982 non-null object
avg_sA                   689982 non-null float64
exhaustion               689982 non-null float64
zone                     689982 non-null object
sector                   689982 non-null object
daily_total_extracted    689982 non-null int64
ep_daily_pct             689982 non-null float64
ep_datestring            689982 non-null object
dtypes: float64(4), int64(4), object(5)
memory usage: 68.4+ MB


In [3]:
date_month = []
date_year = []
datestrings = exhist.datestring.astype(str)
for i in range(len(datestrings)):
    date_month.append(datestrings[i][4:6])
    date_year.append(datestrings[i][:4])
exhist['month'] = date_month
exhist['year'] = date_year
exhist['year_month'] = exhist['year']+exhist['month']
exhist.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct,ep_datestring,month,year,year_month
0,611703,405,20151231,405,76860.0,7700SW,0.202857,0.526932,611,703,34288,1.181171,61170320151231,12,2015,201512
1,611703,424,20160105,829,76860.0,7700SW,0.202857,1.078584,611,703,28457,1.489967,61170320160105,01,2016,201601
2,611703,463,20160128,1292,76860.0,7700SW,0.202857,1.680978,611,703,22431,2.064108,61170320160128,01,2016,201601
3,611703,81,20160223,1373,76860.0,7700SW,0.202857,1.786365,611,703,20923,0.387134,61170320160223,02,2016,201602
4,611703,54,20160224,1427,76860.0,7700SW,0.202857,1.856622,611,703,20941,0.257867,61170320160224,02,2016,201602


In [4]:
ep_region = []
for i in range(len(exhist)):
    if exhist['region'][i] == '7210A':
        ep_region.append('rU')
    elif exhist['region'][i] == '7210M':
        ep_region.append('rV')
    elif exhist['region'][i] == '7210N':
        ep_region.append('rW')
    elif exhist['region'][i] == '7700NW':
        ep_region.append('rY')
    elif exhist['region'][i] == '7700SW':
        ep_region.append('rZ')
    else:
        ep_region.append('rX')
len(ep_region)

689982

In [5]:
histdf = exhist.copy()
histdf['region'] = ep_region
histdf.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct,ep_datestring,month,year,year_month
0,611703,405,20151231,405,76860.0,rZ,0.202857,0.526932,611,703,34288,1.181171,61170320151231,12,2015,201512
1,611703,424,20160105,829,76860.0,rZ,0.202857,1.078584,611,703,28457,1.489967,61170320160105,01,2016,201601
2,611703,463,20160128,1292,76860.0,rZ,0.202857,1.680978,611,703,22431,2.064108,61170320160128,01,2016,201601
3,611703,81,20160223,1373,76860.0,rZ,0.202857,1.786365,611,703,20923,0.387134,61170320160223,02,2016,201602
4,611703,54,20160224,1427,76860.0,rZ,0.202857,1.856622,611,703,20941,0.257867,61170320160224,02,2016,201602


In [6]:
# adding wtd averages for summations in grouping
histdf['average_exhaustion'] = histdf['exhaustion']*histdf['extraction_tons']/histdf['daily_total_extracted']
histdf['ep_sA_reserves'] = histdf['avg_sA']*histdf['reserves']
histdf['region_zone'] = histdf['region']+histdf['zone']

In [7]:
histdf.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct,ep_datestring,month,year,year_month,average_exhaustion,ep_sA_reserves,region_zone
0,611703,405,20151231,405,76860.0,rZ,0.202857,0.526932,611,703,34288,1.181171,61170320151231,12,2015,201512,0.006224,15591.600011,rZ611
1,611703,424,20160105,829,76860.0,rZ,0.202857,1.078584,611,703,28457,1.489967,61170320160105,01,2016,201601,0.016071,15591.600011,rZ611
2,611703,463,20160128,1292,76860.0,rZ,0.202857,1.680978,611,703,22431,2.064108,61170320160128,01,2016,201601,0.034697,15591.600011,rZ611
3,611703,81,20160223,1373,76860.0,rZ,0.202857,1.786365,611,703,20923,0.387134,61170320160223,02,2016,201602,0.006916,15591.600011,rZ611
4,611703,54,20160224,1427,76860.0,rZ,0.202857,1.856622,611,703,20941,0.257867,61170320160224,02,2016,201602,0.004788,15591.600011,rZ611


In [13]:
month_days_total = pd.read_csv('Resources/monthly_composite.csv',index_col=False)
month_days = month_days_total[['year_month','run_days']]
month_days.to_csv('Resources/month_days.csv',index=False)

In [9]:
def ep_agg(x):
    column_names = {
        'zone_total_extracted': x['total_extraction_tons'].sum(),
        'zone_reserves': x['reserves'].sum(),
        'zone_percent_extracted_total': x['ep_daily_pct'].sum(),
        'ep_sA_reserves': x['ep_sA_reserves'].sum()
    }
    return pd.Series(column_names,index=['zone_total_extracted','zone_reserves','zone_percent_extracted_total','ep_sA_reserves'])
zone_months = histdf.groupby(['zone','year_month']).apply(ep_agg).reset_index(drop=False)
zone_months

,zone,year_month,zone_total_extracted,zone_reserves,zone_percent_extracted_total,ep_sA_reserves
0,611,201501,4003.0,329400.0,7.977742,41833.800000
1,611,201503,2703.0,307440.0,7.013188,37573.560022
2,611,201504,40872.0,1866600.0,2.776994,237058.200000
3,611,201505,59036.0,2108160.0,4.979109,266967.720044
4,611,201506,95902.0,2712060.0,14.274692,341763.479967
...,...,...,...,...,...,...
2735,P93,201911,625860.0,647991.0,2.746862,107907.178000
2736,P93,202001,1155.0,439650.0,3.811359,79972.497000
2737,P93,202002,427252.0,1285275.0,34.410272,223002.626912
2738,P93,202003,2437819.0,4206907.0,10.423492,713909.288780


In [10]:
zone_months['zone_exhaustion'] = zone_months['zone_total_extracted']/zone_months['zone_reserves']*100
zone_months['zone_avg_sA'] = zone_months['ep_sA_reserves']/zone_months['zone_reserves']
zone_months.head(10)

,zone,year_month,zone_total_extracted,zone_reserves,zone_percent_extracted_total,ep_sA_reserves,zone_exhaustion,zone_avg_sA
0,611,201501,4003.0,329400.0,7.977742,41833.800000,1.215240,0.127000
1,611,201503,2703.0,307440.0,7.013188,37573.560022,0.879196,0.122214
2,611,201504,40872.0,1866600.0,2.776994,237058.200000,2.189650,0.127000
3,611,201505,59036.0,2108160.0,4.979109,266967.720044,2.800357,0.126635
4,611,201506,95902.0,2712060.0,14.274692,341763.479967,3.536131,0.126016
5,611,201507,143973.0,3579480.0,13.314242,455604.120044,4.022176,0.127282
6,611,201508,78776.0,2119140.0,8.483291,262081.620066,3.717357,0.123674
7,611,201509,149937.0,4655520.0,24.147390,573134.040088,3.220628,0.123108
8,611,201510,281933.0,6017040.0,25.884411,748440.719989,4.685576,0.124387
9,611,201511,286364.0,6104786.0,28.440931,759014.536012,4.690811,0.124331


In [11]:
zone_months.to_csv('Resources/zone_months.csv',index=False)